In [1]:
install.packages("remotes")
remotes::install_github("cellgeni/sceasy")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done




rprojroot  (2.0.3  -> 2.0.4 ) [CRAN]
Rcpp       (1.0.11 -> 1.0.12) [CRAN]
jsonlite   (1.8.7  -> 1.8.8 ) [CRAN]
reticulate (1.32.0 -> 1.35.0) [CRAN]


Installing 4 packages: rprojroot, Rcpp, jsonlite, reticulate

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



── R CMD build ─────────────────────────────────────────────────────────────────
✔  checking for file ‘/tmp/RtmpC2pEBN/remotes42d4398d9047/cellgeni-sceasy-c1c0bf9/DESCRIPTION’
─  preparing ‘sceasy’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘sceasy_0.0.7.tar.gz’
   


In [1]:
library(sceasy)
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(H5weaver))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(stringr))
library(parallel)

Loading required package: reticulate



In [167]:
# # Helper function to compare two folder for similar/different files

# # Specify the paths of the two folders to compare
# folder1 <- "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female"
# folder2 <- "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Male_v2"

# # List files in folder1
# files1 <- dir(folder1)

# # List files in folder2
# files2 <- dir(folder2)

# # Find matching files
# matching_files <- intersect(files1, files2)

# # Find files in folder1 that are not in folder2
# files_only_in_folder1 <- setdiff(files1, files2)

# # Find files in folder2 that are not in folder1
# files_only_in_folder2 <- setdiff(files2, files1)

# # Print results
# cat("Files matching:\n")
# print(matching_files)

# cat("\nFiles only in folder1:\n")
# print(files_only_in_folder1)

# cat("\nFiles only in folder2:\n")
# print(files_only_in_folder2)

Files matching:
 [1] "Adaptive NK cell_Y1D0.csv"                    
 [2] "ASDC_Y1D0.csv"                                
 [3] "BaEoMaP cell_Y1D0.csv"                        
 [4] "C1Q+ CD16 monocyte_Y1D0.csv"                  
 [5] "CD14+ cDC2_Y1D0.csv"                          
 [6] "CD27- effector B cell_Y1D0.csv"               
 [7] "CD27+ effector B cell_Y1D0.csv"               
 [8] "CD4 MAIT_Y1D0.csv"                            
 [9] "CD56bright NK cell_Y1D0.csv"                  
[10] "CD8 MAIT_Y1D0.csv"                            
[11] "CD8aa_Y1D0.csv"                               
[12] "CD95 memory B cell_Y1D0.csv"                  
[13] "cDC1_Y1D0.csv"                                
[14] "CLP cell_Y1D0.csv"                            
[15] "CM CD4 T cell_Y1D0.csv"                       
[16] "CM CD8 T cell_Y1D0.csv"                       
[17] "CMP cell_Y1D0.csv"                            
[18] "Core CD14 monocyte_Y1D0.csv"                  
[19] "Core CD16 monocyte_Y1D0.

In [2]:
meta_data=read.csv('/home/jupyter/IH-A-Aging-Analysis-Notebooks/Qiuyu-Notebooks/scRNA/hise_meta_data_2024-01-23_fixed.csv')
meta_data<-meta_data %>% filter(Covid_exlcusion=='no',sample.visitName %in% c('Flu Year 1 Day 0'))
meta_data<-meta_data %>% mutate (Flu_Year=case_when(Sample_Draw_date>as.Date('2019-6-30')&Sample_Draw_date<as.Date('2020-6-29')~'2019',
                                         Sample_Draw_date>as.Date('2020-6-30')&Sample_Draw_date<as.Date('2021-6-29')~'2020',
                                         Sample_Draw_date>as.Date('2021-6-30')&Sample_Draw_date<as.Date('2022-6-29')~'2021',
                                        ))

In [3]:
t(table(meta_data$subject.subjectGuid,meta_data$Flu_Year))

      
       BR1001 BR1002 BR1003 BR1004 BR1005 BR1006 BR1007 BR1008 BR1009 BR1010
  2019      1      1      1      1      1      1      1      1      1      1
  2020      0      0      0      0      0      0      0      0      0      0
      
       BR1011 BR1012 BR1013 BR1014 BR1015 BR1016 BR1017 BR1018 BR1019 BR1021
  2019      1      1      1      1      0      0      0      1      0      0
  2020      0      0      0      0      1      1      1      0      1      1
      
       BR1023 BR1024 BR1025 BR1026 BR1028 BR1030 BR1031 BR1032 BR1033 BR1037
  2019      0      0      0      0      0      0      0      0      1      0
  2020      1      1      1      1      1      1      1      1      0      1
      
       BR1040 BR1041 BR1042 BR1043 BR1044 BR1045 BR1047 BR1048 BR1049 BR1050
  2019      0      0      0      0      0      0      0      0      0      0
  2020      1      1      1      1      1      1      1      1      1      1
      
       BR1051 BR1052 BR1054 BR1056 BR1057

In [161]:
## add an additional column
meta_data$facet = paste(meta_data$subject.biologicalSex, meta_data$cohort.cohortGuid, sep = "_")
meta_data$facet_2 <- paste(meta_data$facet, meta_data$CMV.IgG.Serology.Result.Interpretation, sep = "_")

In [162]:
head(meta_data)

,X.1,X,lastUpdated,sample.id,sample.bridgingControl,sample.sampleKitGuid,sample.visitName,sample.visitDetails,sample.drawDate,sample.daysSinceFirstVisit,⋯,Sample_Draw_date,Covid_Dose_1,Covid_Dose_2,Covid_exlcusion,BMI,file.name.downloaded,file.path,Flu_Year,facet,facet_2
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,1,2023-11-18T02:03:08.347Z,f499ff83-e513-4d24-a10f-151348269fff,false,KT00001,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-15,NA,NA,no,<25-BMI,B001-P1_PB00001-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/fec489f9-9a74-4635-aa91-d2bf09d1faec/B001-P1_PB00001-01_labeled.h5,2019,Female_BR1,Female_BR1_Negative
2,2,2,2023-11-18T02:03:08.347Z,750e90a9-a296-4f0f-969f-60225c2bca17,false,KT00002,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-15,2020-12-28,2021-01-18,no,<25-BMI,B001-P1_PB00002-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/7c0c7979-eebd-4aba-b5b2-6e76b4643623/B001-P1_PB00002-01_labeled.h5,2019,Male_BR1,Male_BR1_Negative
3,3,3,2023-11-18T02:03:08.347Z,2db6fb3f-e3f4-454b-891b-9b068541b51d,false,KT00003,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-15,2020-12-28,2021-01-16,no,<25-BMI,B001-P1_PB00003-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/40efd03a-cb2f-4677-af42-a056cbfe5a17/B001-P1_PB00003-01_labeled.h5,2019,Female_BR1,Female_BR1_Negative
4,4,4,2023-11-19T10:34:34.12Z,f04693c5-563c-4b5b-ae58-877d0d9ae2fe,false,KT00004,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-21,2021-04-16,2021-05-06,no,<25-BMI,B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/68fbcd34-1d63-461d-8195-df5b8dc61b31/B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5,2019,Male_BR1,Male_BR1_Negative
5,5,5,2023-11-19T05:07:36.805Z,eb5b3a3d-002e-40a6-aa19-aa0e6a7fff8f,false,KT00006,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-23,2021-01-16,2021-02-26,no,<25-BMI,B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/ea8d98e9-e99e-4dc6-9e78-9866e0deac68/B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5,2019,Female_BR1,Female_BR1_Negative
6,6,6,2023-11-18T02:03:08.347Z,f32ac061-0ede-4040-8d8c-35229ea5229c,false,KT00010,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-28,2021-03-21,2021-04-18,no,>25-30BMI,B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/237e41df-5616-4852-8a8f-b32835c9efcb/B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5,2019,Male_BR1,Male_BR1_Negative


# Year 1 Day 0, compare male vs female

### Y1

In [3]:
meta_data_subset<-meta_data%>% filter(sample.visitName %in% c('Flu Year 1 Day 0'))

In [4]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

   user  system elapsed 
835.183 584.184 187.475 

In [7]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  subject.biologicalSex+
                                          cohort.cohortGuid+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("subject.biologicalSex", "Male", "Female")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'Female',log2FoldChange>0~'Male'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/formal_BRIanalysis/DEG_Mehul/Sex_Year1Day0/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



    user   system  elapsed 
1401.875 1987.583 1185.011 

# Compare Flu year 1 day 0

In [9]:
meta_data_subset<-meta_data%>% filter(sample.visitName %in% c('Flu Year 1 Day 0', 'Flu Year 1 Day 7'))

In [10]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

    user   system  elapsed 
1460.701  739.364  109.302 

In [13]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/filtered_gene_list_for_flu_yr1_d0d7_BR1vsBR2.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                               design = ~ sample.visitName + subject.biologicalSex + cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
    
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 1 Day 0", "Flu Year 1 Day 7")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInD7',log2FoldChange>0~'HigherInD0'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/formal_BRIanalysis/DEG_Mehul/Flu_d0_vs_d7_all/",celltype,"_Y1.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

    user   system  elapsed 
3248.990 3951.385 2089.609 

# Compare Age (BR1 vs BR2)

In [16]:
meta_data_subset<-meta_data%>% filter(sample.visitName %in% c('Flu Year 1 Day 0'))

In [17]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

   user  system elapsed 
760.119 445.936  27.621 

In [21]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/filtered_gene_list_for_immunevar_day0_7.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                               design = ~ cohort.cohortGuid + subject.biologicalSex +
                                          CMV.IgG.Serology.Result.Interpretation)
    
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR1", "BR2")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInBR2',log2FoldChange>0~'HigherInBR1'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/formal_BRIanalysis/DEG_Mehul/Age_day_0/",celltype,"_Y1.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



    user   system  elapsed 
1367.510 2565.881 1427.116 

# Compare CMV

In [24]:
meta_data_subset<-meta_data%>% filter(sample.visitName %in% c('Flu Year 1 Day 0'))
meta_data_subset <- meta_data_subset %>% filter(CMV.IgG.Serology.Result.Interpretation %in% c("Negative", "Positive"))

In [25]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

   user  system elapsed 
796.062 524.150  29.788 

In [26]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/filtered_gene_list_for_immunevar_day0_7.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                               design = ~ CMV.IgG.Serology.Result.Interpretation +
                                          cohort.cohortGuid + subject.biologicalSex)
    
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("CMV.IgG.Serology.Result.Interpretation", "Negative", "Positive")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInPositive',log2FoldChange>0~'HigherInNegative'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/formal_BRIanalysis/DEG_Mehul/CMV_yr1_day0/",celltype,"_Y1.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



    user   system  elapsed 
1376.437 2617.381 1464.978 

# Comapre Flu day 0 vs Day 7 in BR1 and BR2 separately

In [14]:
## did it for BR1 and BR2 separately

In [15]:
meta_data_subset<-meta_data%>% filter(cohort.cohortGuid=='BR2', sample.visitName %in% c('Flu Year 1 Day 0','Flu Year 1 Day 7'))

In [18]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

Timing stopped at: 2.607 3.608 6.955



In [ ]:
bulk_list$

In [13]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/filtered_gene_list_for_flu_yr1_d0d7_BR1vsBR2.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName + subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 1 Day 0", "Flu Year 1 Day 7")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInD7',log2FoldChange>0~'HigherInD0'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/formal_BRIanalysis/DEG_Mehul/DESeq2_D0_VS_D7_L3_BR2/",celltype,"_Y1_BR2.csv"))
}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

    user   system  elapsed 
1299.931 2211.108 1241.723 

# Comapre male vs female in BR1 and BR2 separately

In [31]:
## did it for BR1 and BR2 separately

In [2]:
meta_data_subset<-meta_data%>% filter(cohort.cohortGuid=='BR1', sample.visitName %in% c('Flu Year 1 Day 0','Flu Year 1 Day 7'))

ERROR: Error in eval(expr, envir, enclos): object 'meta_data' not found


In [33]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

   user  system elapsed 
477.262 332.912 102.310 

In [34]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/filtered_gene_list_for_sex.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("subject.biologicalSex", "Male", "Female")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'Female',log2FoldChange>0~'Male'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/formal_BRIanalysis/DEG_Mehul/DESeq2_Sex_yr1_Day_BR2/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



    user   system  elapsed 
 815.505 1176.578  665.116 

# Comapre BR1 vs BR2 in Male and Female separately

In [17]:
## did it for Female and Male separately

## Male run

In [4]:
meta_data_subset<-meta_data%>% filter(subject.biologicalSex=='Male', sample.visitName %in% c('Flu Year 1 Day 0')) 
head(meta_data_subset)

,X.1,X,lastUpdated,sample.id,sample.bridgingControl,sample.sampleKitGuid,sample.visitName,sample.visitDetails,sample.drawDate,sample.daysSinceFirstVisit,⋯,pbmc_sample_id,Sample_ID,Sample_Draw_date,Covid_Dose_1,Covid_Dose_2,Covid_exlcusion,BMI,file.name.downloaded,file.path,Flu_Year
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2,2,2023-11-18T02:03:08.347Z,750e90a9-a296-4f0f-969f-60225c2bca17,false,KT00002,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,PB00002-01,NA,2019-10-15,2020-12-28,2021-01-18,no,<25-BMI,B001-P1_PB00002-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/7c0c7979-eebd-4aba-b5b2-6e76b4643623/B001-P1_PB00002-01_labeled.h5,2019
2,4,4,2023-11-19T10:34:34.12Z,f04693c5-563c-4b5b-ae58-877d0d9ae2fe,false,KT00004,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,PB00004-01,NA,2019-10-21,2021-04-16,2021-05-06,no,<25-BMI,B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/68fbcd34-1d63-461d-8195-df5b8dc61b31/B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5,2019
3,6,6,2023-11-18T02:03:08.347Z,f32ac061-0ede-4040-8d8c-35229ea5229c,false,KT00010,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,PB00010-02,NA,2019-10-28,2021-03-21,2021-04-18,no,>25-30BMI,B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/237e41df-5616-4852-8a8f-b32835c9efcb/B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5,2019
4,8,8,2023-11-19T08:45:57.633Z,77c1dad3-605d-4f94-b163-8be9c62d3bc5,false,KT00014,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,PB00014-01,NA,2019-10-30,2021-02-09,2021-03-06,no,<25-BMI,B002-P1_PB00014-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/fbcbf51d-90f8-46ff-bd21-2f56d65306ad/B002-P1_PB00014-01_2023-11-17T21:36:51.794326181Z_labeled.h5,2019
5,9,9,2023-11-19T07:46:30.259Z,c9be600f-50b8-48cf-9d8a-b4d84b0eba6f,false,KT00015,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-11-01T00:00:00Z,0,⋯,PB00015-01,NA,2019-11-01,2021-04-01,2021-04-21,no,>25-30BMI,B002-P1_PB00015-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/119c13a2-9d5a-4c39-89f4-8c7997d30903/B002-P1_PB00015-01_2023-11-17T21:36:51.794326181Z_labeled.h5,2019
6,13,13,2023-11-18T02:03:08.347Z,fc01c04c-7213-471f-8510-1785f9dee039,false,KT00025,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-11-01T00:00:00Z,0,⋯,PB00025-04,NA,2019-11-25,2021-01-29,2021-02-19,no,<25-BMI,B078-P2_PB00025-04_2021-08-19T17:09:29.934849811Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/247d580e-7b47-4f59-bc37-6e29188b7aff/B078-P2_PB00025-04_2021-08-19T17:09:29.934849811Z_labeled.h5,2019


In [5]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

# Combine all pseudobulk objects into a single data frame
master_bulk <- do.call(cbind, bulk_list)

# Save master pseudobulk object
#saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Male_v2/master_male_pseudobulk_test1.rds")

   user  system elapsed 
461.440 293.458  87.291 

In [6]:
# # Combine all pseudobulk objects into a single data frame
# master_bulk <- do.call(cbind, bulk_list)

# # Save master pseudobulk object
# saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/master_pseudobulk.rds")

In [17]:
dict = read.csv("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/Immune_Cell_Order_forDEGsummary - Sheet1.csv")
#dict
celltypes <- list(dict$FINAL.ORDER)
celltypes

[[1]]
 [1] "SOX4+ naive CD4 T cell"              
 [2] "Core naive CD4 T cell"               
 [3] "ISG+ naive CD4 T cell"               
 [4] "CM CD4 T cell"                       
 [5] "GZMB- CD27+ EM CD4 T cell"           
 [6] "GZMB- CD27- EM CD4 T cell"           
 [7] "ISG+ memory CD4 T cell"              
 [8] "KLRF1- GZMB+ CD27- memory CD4 T cell"
 [9] "Naive CD4 Treg"                      
[10] "Memory CD4 Treg"                     
[11] "KLRB1+ memory CD4 Treg"              
[12] "GZMK+ memory CD4 Treg"               
[13] "SOX4+ naive CD8 T cell"              
[14] "Core naive CD8 T cell "              
[15] "ISG+ naive CD8 T cell"               
[16] "CM CD8 T cell"                       
[17] "GZMK+ CD27+ EM CD8 T cell"           
[18] "KLRF1- GZMB+ CD27- EM CD8 T cell"    
[19] "GZMK- CD27+ EM CD8 T cell"           
[20] "KLRF1+ GZMB+ CD27- EM CD8 T cell"    
[21] "ISG+ memory CD8 T cell"              
[22] "Memory CD8 Treg"                     
[23] "KLRB1+ memory CD8 Treg"              
[24] "CD8aa T cell"                        
[25] "DN T cell"                           
[26] "CD8 MAIT"                            
[27] "CD4 MAIT"                            
[28] "ISG+ MAIT"                           
[29] "SOX4+ Vd1 gdT"                       
[30] "Naive Vd1 gdT"                       
[31] "KLRF1- effector Vd1 gdT"             
[32] "KLRF1+ effector Vd1 gdT"             
[33] "GZMK+ Vd2 gdT"                       
[34] "GZMB+ Vd2 gdT"                       
[35] "Proliferating T cell"                
[36] "Transitional B cell"                 
[37] "Core naive B cell"                   
[38] "ISG+ naive B cell"                   
[39] "Early memory B cell"                 
[40] "Core memory B cell"                  
[41] "Activated memory B cell"             
[42] "Type 2 polarized memory B cell"      
[43] "CD95 memory B cell"                  
[44] "CD27+ effector B cell"               
[45] "CD27- effector B cell"               
[46] "Plasma cell"                         
[47] "GZMK- CD56dim NK cell"               
[48] "GZMK+ CD56dim NK cell"               
[49] "ISG+ CD56dim NK cell"                
[50] "CD56bright NK cell"                  
[51] "Adaptive NK cell"                    
[52] "Proliferating NK cell"               
[53] "Core CD14 monocyte"                  
[54] "ISG+ CD14 monocyte"                  
[55] "IL1B+ CD14 monocyte"                 
[56] "Intermediate monocyte"               
[57] "Core CD16 monocyte"                  
[58] "ISG+ CD16 monocyte"                  
[59] "C1Q+ CD16 monocyte"                  
[60] "ASDC"                                
[61] "cDC1"                                
[62] "CD14+ cDC2"                          
[63] "HLA-DRhi cDC2"                       
[64] "ISG+ cDC2"                           
[65] "pDC"                                 
[66] "ILC"                                 
[67] "Erythrocyte"                         
[68] "Platelet"                            
[69] "CMP cell"                            
[70] "CLP cell"                            
[71] "BaEoMaP cell"

In [20]:
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
head(selected_gene_list)

,X,gene,AIFI_L3
,<int>,<chr>,<chr>
1,0,AL669831.5,cDC1
2,1,NOC2L,cDC1
3,2,ISG15,cDC1
4,3,SDF4,cDC1
5,4,B3GALT6,cDC1
6,5,UBE2J2,cDC1


In [21]:
system.time(
for (celltype in unique(selected_gene_list$AIFI_L3)){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Male_v2/",celltype,"_Y1D0.csv"))

}
)

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "KLRF1- effector Vd1 gdT"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "ISG+ CD16 monocyte"


...

[1] "DN T cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



    user   system  elapsed 
 969.299 1196.817  802.016 

In [19]:
#celltype = "ISG+ MAIT"


system.time(
for (cell in celltype){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Male_v3/",celltype,"_Y1D0.csv"))

}
)

 [1] "SOX4+ naive CD4 T cell"              
 [2] "Core naive CD4 T cell"               
 [3] "ISG+ naive CD4 T cell"               
 [4] "CM CD4 T cell"                       
 [5] "GZMB- CD27+ EM CD4 T cell"           
 [6] "GZMB- CD27- EM CD4 T cell"           
 [7] "ISG+ memory CD4 T cell"              
 [8] "KLRF1- GZMB+ CD27- memory CD4 T cell"
 [9] "Naive CD4 Treg"                      
[10] "Memory CD4 Treg"                     
[11] "KLRB1+ memory CD4 Treg"              
[12] "GZMK+ memory CD4 Treg"               
[13] "SOX4+ naive CD8 T cell"              
[14] "Core naive CD8 T cell "              
[15] "ISG+ naive CD8 T cell"               
[16] "CM CD8 T cell"                       
[17] "GZMK+ CD27+ EM CD8 T cell"           
[18] "KLRF1- GZMB+ CD27- EM CD8 T cell"    
[19] "GZMK- CD27+ EM CD8 T cell"           
[20] "KLRF1+ GZMB+ CD27- EM CD8 T cell"    
[21] "ISG+ memory CD8 T cell"              
[22] "Memory CD8 Treg"                     
[23] "KLRB1+ memory CD8 Treg"   

Joining with `by = join_by(pbmc_sample_id)`
Warning message:
“There was 1 warning in `filter()`.
ℹ In argument: `AIFI_L3 == celltype`.
Caused by warning in `AIFI_L3 == celltype`:
! longer object length is not a multiple of shorter object length”
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



ERROR: Error in `$<-.data.frame`(`*tmp*`, "celltype", value = c("SOX4+ naive CD4 T cell", : replacement has 71 rows, data has 5182


Timing stopped at: 11.78 14.44 10.65



In [11]:
print(res)

                 baseMean log2FoldChange      lfcSE          stat      pvalue
NOC2L          0.41670470   0.2997048854 0.74859473  0.4003566584 0.688893842
ISG15          5.19528753  -0.5840641135 0.27344227 -2.1359686412 0.032681970
SDF4           0.66703853  -0.0307356057 0.55448589 -0.0554308165 0.955795246
UBE2J2         0.89414348   0.1215124252 0.52211234  0.2327323385 0.815969256
INTS11         0.43746968  -0.7211936056 0.63394271 -1.1376321438 0.255274119
AURKAIP1       1.78043645   0.1289834814 0.35107779  0.3673928869 0.713325977
CCNL2          0.90358588  -0.3839726348 0.54824827 -0.7003626929 0.483700829
MRPL20         2.84134348   0.0253952280 0.37116691  0.0684199670 0.945451328
SSU72          1.96031319  -0.2484132770 0.36166269 -0.6868645381 0.492168096
MIB2           0.48671415  -0.3151524840 0.58842520 -0.5355863184 0.592244486
CDK11B         0.51660372  -0.4678397717 0.56828551 -0.8232477567 0.410367116
CDK11A         0.81955096  -0.4249992340 0.48676308 -0.873113124

## Test 1 (random selection)- only in Females

In [28]:
meta_data_subset<-meta_data%>% filter(subject.biologicalSex=='Female', sample.visitName %in% c('Flu Year 1 Day 0')) 
head(meta_data_subset)

,X.1,X,lastUpdated,sample.id,sample.bridgingControl,sample.sampleKitGuid,sample.visitName,sample.visitDetails,sample.drawDate,sample.daysSinceFirstVisit,⋯,Sample_ID,Sample_Draw_date,Covid_Dose_1,Covid_Dose_2,Covid_exlcusion,BMI,file.name.downloaded,file.path,Flu_Year,facet
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,1,2023-11-18T02:03:08.347Z,f499ff83-e513-4d24-a10f-151348269fff,false,KT00001,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,NA,2019-10-15,NA,NA,no,<25-BMI,B001-P1_PB00001-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/fec489f9-9a74-4635-aa91-d2bf09d1faec/B001-P1_PB00001-01_labeled.h5,2019,Female_BR1
2,3,3,2023-11-18T02:03:08.347Z,2db6fb3f-e3f4-454b-891b-9b068541b51d,false,KT00003,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,NA,2019-10-15,2020-12-28,2021-01-16,no,<25-BMI,B001-P1_PB00003-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/40efd03a-cb2f-4677-af42-a056cbfe5a17/B001-P1_PB00003-01_labeled.h5,2019,Female_BR1
3,5,5,2023-11-19T05:07:36.805Z,eb5b3a3d-002e-40a6-aa19-aa0e6a7fff8f,false,KT00006,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,NA,2019-10-23,2021-01-16,2021-02-26,no,<25-BMI,B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/ea8d98e9-e99e-4dc6-9e78-9866e0deac68/B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5,2019,Female_BR1
4,7,7,2023-11-19T10:06:55.06Z,fb3c3406-2797-4c31-b085-37d8138099a3,false,KT00012,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,NA,2019-10-29,NA,NA,no,>30BMI,B002-P1_PB00012-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/7a99c4c8-5438-430a-a37a-5b5f4052c064/B002-P1_PB00012-01_2023-11-17T21:36:51.794326181Z_labeled.h5,2019,Female_BR1
5,10,10,2023-11-19T09:36:51.309Z,19e5fa0e-52a8-4bef-a19a-075ad37fd5e3,false,KT00016,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-11-01T00:00:00Z,0,⋯,NA,2019-11-04,2021-04-07,2021-04-23,no,<25-BMI,B002-P1_PB00016-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/d22d313f-b908-416b-aa78-b534fcce758c/B002-P1_PB00016-01_2023-11-17T21:36:51.794326181Z_labeled.h5,2019,Female_BR2
6,11,11,2023-11-19T12:10:10.248Z,dedcaa14-0aa3-403b-9e13-475e715f0d69,false,KT00022,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-11-01T00:00:00Z,0,⋯,NA,2019-11-13,2021-03-18,2021-04-15,no,<25-BMI,B002-P2_PB00022-01_2023-11-17T21:38:04.103392546Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/9114124c-af57-47a7-bde3-ade4a150ac82/B002-P2_PB00022-01_2023-11-17T21:38:04.103392546Z_labeled.h5,2019,Female_BR1


In [77]:
# Filter the data based on the criteria
subset_BR2 <- filter(meta_data_subset, facet == "Female_BR2")
subset_BR1 <- filter(meta_data_subset, facet == "Female_BR1")

length(subset_BR2$pbmc_sample_id)


[1] 20

In [78]:
# Sample from the filtered subsets
selected_sample_ids_BR2 <- sample(unique(subset_BR2$pbmc_sample_id), 20)
selected_sample_ids_BR1 <- sample(unique(subset_BR1$pbmc_sample_id), 21)
length(selected_sample_ids_BR2)
length(selected_sample_ids_BR1)
#length(unique(selected_sample_ids_BR1$sample.id))
selected_sample_ids_BR1

[1] 20

[1] 21

[1] "PB00350-01" "PB00621-01" "PB00347-01" "PB00030-02" "PB00645-01"
 [6] "PB00341-01" "PB00362-01" "PB00012-01" "PB01425-01" "PB00001-01"
[11] "PB00031-05" "PB00575-01" "PB00507-01" "PB00365-01" "PB00366-01"
[16] "PB00338-01" "PB00003-01" "PB00023-05" "PB00529-01" "PB00525-01"
[21] "PB00349-01"

In [79]:
# Combine the sampled IDs
selected_sample_ids <- c(selected_sample_ids_BR2, selected_sample_ids_BR1)
#length(selected_sample_ids$facet)
length(selected_sample_ids)

[1] 41

In [80]:
# #head(selected_sample_ids)
# length(unique(subset_BR2$pbmc_sample_id))
# length(unique(selected_sample_ids))
# #selected_sample_ids_BR2

In [81]:
# Filter the data for the selected sample IDs
meta_data_subset <- meta_data_subset[meta_data_subset$pbmc_sample_id %in% selected_sample_ids, ]
length(unique(meta_data_subset$pbmc_sample_id))

[1] 41

In [82]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

# Combine all pseudobulk objects into a single data frame
master_bulk <- do.call(cbind, bulk_list)

# Save master pseudobulk object
saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_Test_1/master_female_pseudobulk_test1.rds")

   user  system elapsed 
509.647 348.802  24.687 

In [6]:
# # Combine all pseudobulk objects into a single data frame
# master_bulk <- do.call(cbind, bulk_list)

# # Save master pseudobulk object
# saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/master_pseudobulk.rds")

In [83]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_Test_1/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 3 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 3 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



    user   system  elapsed 
1034.089 1444.971  869.875 

## Test 2 (random selection of 20 in BR2 and 21 in BR1)- only in Females

In [84]:
meta_data_subset<-meta_data%>% filter(subject.biologicalSex=='Female', sample.visitName %in% c('Flu Year 1 Day 0')) 
#head(meta_data_subset)

In [85]:
# Filter the data based on the criteria
subset_BR2 <- filter(meta_data_subset, facet == "Female_BR2")
subset_BR1 <- filter(meta_data_subset, facet == "Female_BR1")

length(subset_BR2$pbmc_sample_id)


[1] 25

In [86]:
# Sample from the filtered subsets
selected_sample_ids_BR2 <- sample(unique(subset_BR2$pbmc_sample_id), 20)
selected_sample_ids_BR1 <- sample(unique(subset_BR1$pbmc_sample_id), 21)
length(selected_sample_ids_BR2)
length(selected_sample_ids_BR1)
#length(unique(selected_sample_ids_BR1$sample.id))
selected_sample_ids_BR1

[1] 20

[1] 21

[1] "PB00023-05" "PB00001-01" "PB00347-01" "PB00362-01" "PB00529-01"
 [6] "PB00349-01" "PB00365-01" "PB00003-01" "PB00507-01" "PB00341-01"
[11] "PB00350-01" "PB00525-01" "PB00022-01" "PB00012-01" "PB00345-01"
[16] "PB00366-01" "PB00575-01" "PB00621-01" "PB00645-01" "PB00030-02"
[21] "PB00166-01"

In [87]:
# Combine the sampled IDs
selected_sample_ids <- c(selected_sample_ids_BR2, selected_sample_ids_BR1)
#length(selected_sample_ids$facet)
length(selected_sample_ids)

[1] 41

In [80]:
# #head(selected_sample_ids)
# length(unique(subset_BR2$pbmc_sample_id))
# length(unique(selected_sample_ids))
# #selected_sample_ids_BR2

In [88]:
# Filter the data for the selected sample IDs
meta_data_subset <- meta_data_subset[meta_data_subset$pbmc_sample_id %in% selected_sample_ids, ]
length(unique(meta_data_subset$pbmc_sample_id))

[1] 41

In [89]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

# Combine all pseudobulk objects into a single data frame
master_bulk <- do.call(cbind, bulk_list)

# Save master pseudobulk object
saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_Test_2/master_female_pseudobulk_test2.rds")

   user  system elapsed 
490.424 330.616  21.977 

In [6]:
# # Combine all pseudobulk objects into a single data frame
# master_bulk <- do.call(cbind, bulk_list)

# # Save master pseudobulk object
# saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/master_pseudobulk.rds")

In [90]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_Test_2/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



    user   system  elapsed 
 971.021 1591.733  900.132 

## Test 3 (random selection of 20 in BR2 and 21 in BR1)- only in Females

In [91]:
meta_data_subset<-meta_data%>% filter(subject.biologicalSex=='Female', sample.visitName %in% c('Flu Year 1 Day 0')) 
#head(meta_data_subset)

In [92]:
# Filter the data based on the criteria
subset_BR2 <- filter(meta_data_subset, facet == "Female_BR2")
subset_BR1 <- filter(meta_data_subset, facet == "Female_BR1")

length(subset_BR2$pbmc_sample_id)


[1] 25

In [93]:
# Sample from the filtered subsets
selected_sample_ids_BR2 <- sample(unique(subset_BR2$pbmc_sample_id), 20)
selected_sample_ids_BR1 <- sample(unique(subset_BR1$pbmc_sample_id), 21)
length(selected_sample_ids_BR2)
length(selected_sample_ids_BR1)
#length(unique(selected_sample_ids_BR1$sample.id))
selected_sample_ids_BR1

[1] 20

[1] 21

[1] "PB00621-01" "PB00022-01" "PB00525-01" "PB00012-01" "PB00166-01"
 [6] "PB00006-01" "PB01425-01" "PB00347-01" "PB00341-01" "PB00365-01"
[11] "PB00031-05" "PB00507-01" "PB00362-01" "PB00001-01" "PB00349-01"
[16] "PB00529-01" "PB00338-01" "PB00030-02" "PB00023-05" "PB00645-01"
[21] "PB00366-01"

In [94]:
# Combine the sampled IDs
selected_sample_ids <- c(selected_sample_ids_BR2, selected_sample_ids_BR1)
#length(selected_sample_ids$facet)
length(selected_sample_ids)

[1] 41

In [95]:
# #head(selected_sample_ids)
# length(unique(subset_BR2$pbmc_sample_id))
# length(unique(selected_sample_ids))
# #selected_sample_ids_BR2

In [96]:
# Filter the data for the selected sample IDs
meta_data_subset <- meta_data_subset[meta_data_subset$pbmc_sample_id %in% selected_sample_ids, ]
length(unique(meta_data_subset$pbmc_sample_id))

[1] 41

In [97]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

# Combine all pseudobulk objects into a single data frame
master_bulk <- do.call(cbind, bulk_list)

# Save master pseudobulk object
saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_Test_3/master_female_pseudobulk_test3.rds")

   user  system elapsed 
456.523 300.389  22.417 

In [6]:
# # Combine all pseudobulk objects into a single data frame
# master_bulk <- do.call(cbind, bulk_list)

# # Save master pseudobulk object
# saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/master_pseudobulk.rds")

In [98]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_Test_3/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 3 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



    user   system  elapsed 
 981.246 1580.975  944.298 

## Test 1 - with CMV sample size accounted for in Females

In [132]:
meta_data_subset<-meta_data%>% filter(subject.biologicalSex=='Female', sample.visitName %in% c('Flu Year 1 Day 0')) 
#head(meta_data_subset)
dim(meta_data_subset)

[1] 51 39

In [133]:
# Filter the data based on the criteria
subset_BR2 <- filter(meta_data_subset, facet_2 == "Female_BR2_Positive")
subset_BR1 <- filter(meta_data_subset, facet_2 == "Female_BR1_Negative")
subset_BR2_N <- filter(meta_data_subset, facet_2 == "Female_BR2_Negative")
subset_BR1_P <- filter(meta_data_subset, facet_2 == "Female_BR1_Positive")

length(subset_BR2$pbmc_sample_id) + length(subset_BR1$pbmc_sample_id) + length(subset_BR2_N$pbmc_sample_id) +length(subset_BR1_P$pbmc_sample_id)


[1] 51

In [134]:
# Sample from the filtered subsets
selected_sample_ids_BR2 <- sample(unique(subset_BR2$pbmc_sample_id), 11)
selected_sample_ids_BR1 <- sample(unique(subset_BR1$pbmc_sample_id), 12)
selected_BR2_N <- sample(unique(subset_BR2_N$pbmc_sample_id), 9)
selected_BR1_P <- sample(unique(subset_BR1_P$pbmc_sample_id), 9)

length(selected_sample_ids_BR2)
length(selected_sample_ids_BR1)
length(selected_BR2_N)
length(selected_BR1_P)

#length(unique(selected_sample_ids_BR1$sample.id))
selected_sample_ids_BR1
selected_sample_ids_BR2
selected_BR2_N
selected_BR1_P

[1] 11

[1] 12

[1] 9

[1] 9

[1] "PB00362-01" "PB00350-01" "PB00529-01" "PB00621-01" "PB00023-05"
 [6] "PB00003-01" "PB00006-01" "PB00030-02" "PB00347-01" "PB00365-01"
[11] "PB00001-01" "PB00345-01"

[1] "PB00552-01" "PB00569-01" "PB00352-04" "PB00557-01" "PB00533-01"
 [6] "PB00387-01" "PB00599-01" "PB00602-01" "PB00530-01" "PB00537-01"
[11] "PB00368-07"

[1] "PB00353-04" "PB00501-01" "PB00394-02" "PB00526-01" "PB00377-01"
[6] "PB00545-01" "PB00334-01" "PB00568-01" "PB00393-01"

[1] "PB00031-05" "PB00012-01" "PB00349-01" "PB00022-01" "PB00575-01"
[6] "PB00525-01" "PB00341-01" "PB00166-01" "PB00532-01"

In [135]:
# Combine the sampled IDs
selected_sample_ids <- c(selected_sample_ids_BR2, selected_sample_ids_BR1, selected_BR1_P, selected_BR2_N)

length(selected_sample_ids)
length(unique(selected_sample_ids))
#selected_sample_ids[duplicated(selected_sample_ids)] # check for duplicated if number of sample ids not same

[1] 41

[1] 41

character(0)

In [136]:
# #head(selected_sample_ids)
# length(unique(subset_BR2$pbmc_sample_id))
# length(unique(selected_sample_ids))
# #selected_sample_ids_BR2

In [137]:
# Filter the data for the selected sample IDs
meta_data_subset <- meta_data_subset[meta_data_subset$pbmc_sample_id %in% selected_sample_ids, ]
length(unique(meta_data_subset$pbmc_sample_id))
length(unique(meta_data_subset$pbmc_sample_id))

[1] 41

[1] 41

In [138]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

# Combine all pseudobulk objects into a single data frame
master_bulk <- do.call(cbind, bulk_list)

# Save master pseudobulk object
saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_1/master_female_CMV_pseudobulk_tes1.rds")

   user  system elapsed 
494.604 322.614  23.108 

In [6]:
# # Combine all pseudobulk objects into a single data frame
# master_bulk <- do.call(cbind, bulk_list)

# # Save master pseudobulk object
# saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/master_pseudobulk.rds")

In [139]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_1/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 4 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



    user   system  elapsed 
1008.616 1618.449  948.607 

## Test 2 - with CMV sample size accounted for in Females

In [140]:
meta_data_subset<-meta_data%>% filter(subject.biologicalSex=='Female', sample.visitName %in% c('Flu Year 1 Day 0')) 
#head(meta_data_subset)
dim(meta_data_subset)

[1] 51 39

In [142]:
# Filter the data based on the criteria
subset_BR2 <- filter(meta_data_subset, facet_2 == "Female_BR2_Positive")
subset_BR1 <- filter(meta_data_subset, facet_2 == "Female_BR1_Negative")
subset_BR2_N <- filter(meta_data_subset, facet_2 == "Female_BR2_Negative")
subset_BR1_P <- filter(meta_data_subset, facet_2 == "Female_BR1_Positive")

length(unique(subset_BR1_P$pbmc_sample_id))
length(unique(subset_BR2_N$pbmc_sample_id))
length(subset_BR2$pbmc_sample_id) + length(subset_BR1$pbmc_sample_id) + length(subset_BR2_N$pbmc_sample_id) +length(subset_BR1_P$pbmc_sample_id)


[1] 9

[1] 9

[1] 51

In [143]:
# Sample from the filtered subsets
selected_sample_ids_BR2 <- sample(unique(subset_BR2$pbmc_sample_id), 11)
selected_sample_ids_BR1 <- sample(unique(subset_BR1$pbmc_sample_id), 12)
selected_BR2_N <- sample(unique(subset_BR2_N$pbmc_sample_id), 9)
selected_BR1_P <- sample(unique(subset_BR1_P$pbmc_sample_id), 9)

length(selected_sample_ids_BR2)
length(selected_sample_ids_BR1)
length(selected_BR2_N)
length(selected_BR1_P)

#length(unique(selected_sample_ids_BR1$sample.id))
selected_sample_ids_BR1
selected_sample_ids_BR2
selected_BR2_N
selected_BR1_P

[1] 11

[1] 12

[1] 9

[1] 9

[1] "PB00507-01" "PB00003-01" "PB00001-01" "PB00347-01" "PB00338-01"
 [6] "PB00345-01" "PB00365-01" "PB00621-01" "PB00023-05" "PB00529-01"
[11] "PB01425-01" "PB00366-01"

[1] "PB00387-01" "PB00541-01" "PB00557-01" "PB00530-01" "PB00552-01"
 [6] "PB00602-01" "PB00561-04" "PB00516-01" "PB00368-07" "PB00537-01"
[11] "PB00569-01"

[1] "PB00393-01" "PB00334-01" "PB00394-02" "PB00377-01" "PB00501-01"
[6] "PB00545-01" "PB00526-01" "PB00353-04" "PB00568-01"

[1] "PB00012-01" "PB00022-01" "PB00166-01" "PB00575-01" "PB00532-01"
[6] "PB00525-01" "PB00341-01" "PB00349-01" "PB00031-05"

In [144]:
# Combine the sampled IDs
selected_sample_ids <- c(selected_sample_ids_BR2, selected_sample_ids_BR1, selected_BR1_P, selected_BR2_N)

length(selected_sample_ids)
length(unique(selected_sample_ids))
#selected_sample_ids[duplicated(selected_sample_ids)] # check for duplicated if number of sample ids not same

[1] 41

[1] 41

In [136]:
# #head(selected_sample_ids)
# length(unique(subset_BR2$pbmc_sample_id))
# length(unique(selected_sample_ids))
# #selected_sample_ids_BR2

In [145]:
# Filter the data for the selected sample IDs
meta_data_subset <- meta_data_subset[meta_data_subset$pbmc_sample_id %in% selected_sample_ids, ]
length(unique(meta_data_subset$pbmc_sample_id))
length(unique(meta_data_subset$pbmc_sample_id))

[1] 41

[1] 41

In [146]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

# Combine all pseudobulk objects into a single data frame
master_bulk <- do.call(cbind, bulk_list)

# Save master pseudobulk object
saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_2/master_female_CMV_pseudobulk_test2.rds")


   user  system elapsed 
491.104 310.628  21.903 

In [6]:
# # Combine all pseudobulk objects into a single data frame
# master_bulk <- do.call(cbind, bulk_list)

# # Save master pseudobulk object
# saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/master_pseudobulk.rds")

In [147]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_2/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 4 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



    user   system  elapsed 
1000.017 1614.312  966.694 

## Test 3 - with CMV sample size accounted for in Females

In [148]:
meta_data_subset<-meta_data%>% filter(subject.biologicalSex=='Female', sample.visitName %in% c('Flu Year 1 Day 0')) 
#head(meta_data_subset)
dim(meta_data_subset)

[1] 51 39

In [149]:
# Filter the data based on the criteria
subset_BR2 <- filter(meta_data_subset, facet_2 == "Female_BR2_Positive")
subset_BR1 <- filter(meta_data_subset, facet_2 == "Female_BR1_Negative")
subset_BR2_N <- filter(meta_data_subset, facet_2 == "Female_BR2_Negative")
subset_BR1_P <- filter(meta_data_subset, facet_2 == "Female_BR1_Positive")

length(unique(subset_BR1_P$pbmc_sample_id))
length(subset_BR2$pbmc_sample_id) + length(subset_BR1$pbmc_sample_id) + length(subset_BR2_N$pbmc_sample_id) +length(subset_BR1_P$pbmc_sample_id)


[1] 9

[1] 51

In [150]:
# Sample from the filtered subsets
selected_sample_ids_BR2 <- sample(unique(subset_BR2$pbmc_sample_id), 11)
selected_sample_ids_BR1 <- sample(unique(subset_BR1$pbmc_sample_id), 12)
selected_BR2_N <- sample(unique(subset_BR2_N$pbmc_sample_id), 9)
selected_BR1_P <- sample(unique(subset_BR1_P$pbmc_sample_id), 9)

length(selected_sample_ids_BR2)
length(selected_sample_ids_BR1)
length(selected_BR2_N)
length(selected_BR1_P)

#length(unique(selected_sample_ids_BR1$sample.id))
selected_sample_ids_BR1
selected_sample_ids_BR2
selected_BR2_N
selected_BR1_P

[1] 11

[1] 12

[1] 9

[1] 9

[1] "PB00003-01" "PB00366-01" "PB00338-01" "PB00347-01" "PB01425-01"
 [6] "PB00621-01" "PB00345-01" "PB00529-01" "PB00030-02" "PB00362-01"
[11] "PB00023-05" "PB00507-01"

[1] "PB00388-01" "PB00602-01" "PB00537-01" "PB00352-04" "PB00533-01"
 [6] "PB00552-01" "PB00368-07" "PB00016-01" "PB00561-04" "PB00387-01"
[11] "PB00599-01"

[1] "PB00394-02" "PB00526-01" "PB00353-04" "PB00377-01" "PB00545-01"
[6] "PB00393-01" "PB00568-01" "PB00501-01" "PB00334-01"

[1] "PB00166-01" "PB00349-01" "PB00532-01" "PB00022-01" "PB00012-01"
[6] "PB00525-01" "PB00575-01" "PB00341-01" "PB00031-05"

In [152]:
# Combine the sampled IDs
selected_sample_ids <- c(selected_sample_ids_BR2, selected_sample_ids_BR1, selected_BR1_P, selected_BR2_N)

length(selected_sample_ids)
length(unique(selected_sample_ids))
#selected_sample_ids[duplicated(selected_sample_ids)] # check for duplicated if number of sample ids not same

[1] 41

[1] 41

In [153]:
# #head(selected_sample_ids)
# length(unique(subset_BR2$pbmc_sample_id))
# length(unique(selected_sample_ids))
# #selected_sample_ids_BR2

In [154]:
# Filter the data for the selected sample IDs
meta_data_subset <- meta_data_subset[meta_data_subset$pbmc_sample_id %in% selected_sample_ids, ]
length(unique(meta_data_subset$pbmc_sample_id))
length(unique(meta_data_subset$pbmc_sample_id))

[1] 41

[1] 41

In [155]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

# Combine all pseudobulk objects into a single data frame
master_bulk <- do.call(cbind, bulk_list)

# Save master pseudobulk object
saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_3/master_female_CMV_pseudobulk_test3.rds")


   user  system elapsed 
449.516 306.198  23.000 

In [156]:
# # Combine all pseudobulk objects into a single data frame
# master_bulk <- do.call(cbind, bulk_list)

# # Save master pseudobulk object
# saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/master_pseudobulk.rds")

In [157]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_3/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



    user   system  elapsed 
 964.433 1565.503  919.355 

# Comapre Immune variation day 0 vs Day 7

In [54]:
meta_data_subset<-meta_data%>% filter(sample.visitName %in% c('Immune Variation Day 0', 'Immune Variation Day 7'))

In [55]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

   user  system elapsed 
814.608 487.541  87.533 

In [59]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/filtered_gene_list_for_immunevar_day0_7.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName + subject.biologicalSex+
                                          cohort.cohortGuid+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Immune Variation Day 0", "Immune Variation Day 7")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInD7',log2FoldChange>0~'HigherInD0'))
res$Year<-"Immvar"
write.csv(res,paste0("/home/jupyter/formal_BRIanalysis/DEG_Mehul/DESeq1_Immvar/",celltype,"_immvar.csv"))
}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Adaptive NK cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "BaEoMaP cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

    user   system  elapsed 
2422.750 3256.497 1669.742 

# Flu Y1 D0 VS D7---Flu Y1 D0 vs D7 - seperate BR1/BR2

### Y1

In [4]:
for (Cohort in c("BR1","BR2")){
meta_data_subset<-meta_data%>% filter(cohort.cohortGuid==Cohort,sample.visitName %in% c('Flu Year 1 Day 0','Flu Year 1 Day 7'))
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)

})
    
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/filtered_gene_list_for_flu_yr1_d0d7_BR1vsBR2.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName+
                                          subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 1 Day 0", "Flu Year 1 Day 7")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInD7',log2FoldChange>0~'HigherInD0'))
res$Year<-"Y1"
res$cohort.cohortGuid<-Cohort  
write.csv(res,paste0("/home/jupyter/formal_BRIanalysis/DEG_Mehul/DESeq2_D0_VS_D7_L3_by_Cohorts/",celltype,"_Y1_",Cohort,".csv"))
}    
}

Warning message in mclapply(meta_data_subset$pbmc_sample_id, function(x) {:
“all scheduled cores encountered errors in user code”
Warning message in mclapply(meta_data_subset$pbmc_sample_id, function(x) {:
“all scheduled cores encountered errors in user code”


### Y2

In [ ]:
for (Cohort in c("BR1","BR2")){
meta_data_subset<-meta_data%>% filter(cohort.cohortGuid==Cohort,sample.visitName %in% c('Flu Year 2 Day 0','Flu Year 2 Day 7'))
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)

},mc.cores=60)
    
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName+
                                          subject.biologicalSex+cohort.cohortGuid+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 2 Day 0", "Flu Year 2 Day 7")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInD7',log2FoldChange>0~'HigherInD0'))
res$Year<-"Y2"
res$cohort.cohortGuid<-Cohort  
write.csv(res,paste0("01C_DESeq2_D0_VS_D7_L3_by_Cohorts/",celltype,"_Y2_",Cohort,".csv"))

}
    

}